In [5]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import cryptography.exceptions
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.ciphers.aead import AESGCM


In [1]:
class Emitter:
    def __init__(self, password, n):
        self.password = password
        self.words = None
        self.salt = os.urandom(16)
        self.PRG(self.get_seed(password), n)    

    def get_seed(self, password):
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=self.salt,
            iterations=390000,)
        seed = kdf.derive(bytes(password, 'utf-8'))
        return seed
    
    def PRG(self, seed, n): 
        #64 bites = 8 blocos de 1 byte(8 bits)
        #gerador com limite de 2^n palavras 
        dig = hashes.Hash(hashes.SHAKE256(8 * pow(2,n)))
        dig.update(seed)
        rand = dig.finalize()
        self.words = rand

    def authentication(self, message):
        h = hmac.HMAC(self.words, hashes.SHA256())
        h.update(message)
        return h.finalize()

    def encode(self, data):
        #fazer o padding do texto 
        padder = padding.PKCS7(64).padder()
        padded = padder.update(data) + padder.finalize()
        #verificar se a chave tem tamanho suficiente para cifrar o texto dado como input 
        if (len(self.words) < len(padded)):
            raise Exception("O tamanho da chave não é suficiente!")        
        else:
            cipher_text = b''
            #cifrar com as words geradas pelo PRG
            for i in range (0,len(padded),8):
                p = padded[i:i+8]
                for index, block in enumerate(p):  
                    #aplicar as máscaras XOR aos blocos 
                    cipher_text += bytes([block ^ self.words[i*8:(i+1)*8][index]])
            return cipher_text     

       
    def sendMessage(self, mensagem):
        auth = self.authentication(b'This is my signature')
        ciphertext = self.encode(mensagem)
        print("Texto cifrado:")
        print(ciphertext)
        return auth + self.salt + ciphertext


